In [1]:
import tensorflow as tf
# import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import os

2023-05-25 15:52:13.893783: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 15:52:17.476004: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.3/lib64:/scratch/ponel/python/lib:/usr/local/cuda-11.3/tensorRT-8.0/lib64:/usr/local/cuda-11.3/cudnn-8.2/lib64:/usr/local/cuda-11.3/nccl-2.9/lib64
2023-05-25 15:52:17.476276: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: 

In [2]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
!nvidia-smi

Thu May 25 15:52:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:B1:00.0 Off |                  N/A |
| 22%   31C    P2    58W / 250W |    445MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:B2:00.0 Off |                  N/A |
| 22%   

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten

In [4]:
from functools import partial
from sklearn.model_selection import train_test_split

from sklearn import tree
from graphviz import Source

## Load the Dataset

In [5]:
img_dim_x = 28
img_dim_y = 28
img_dim_z = 1

batch_size = 1024

In [6]:
def load_MNIST(dirpath='/data/project/FoolingDetection/mnist.npz'):
    with np.load(dirpath) as data:
        x_train, y_train = data['x_train'], data['y_train']
        x_test, y_test = data['x_test'], data['y_test']

    x_train = x_train/255
    x_test = x_test/255

    x_train = x_train.reshape(-1, 28, 28, 1)
    x_test = x_test.reshape(-1, 28, 28, 1)

    return x_train, y_train, x_test, y_test

In [7]:
def load_CIFAR_batch(filename):
    """Load a single batch of the CIFAR-10 dataset"""
    with open(filename, 'rb') as f:
        datadict = pickle.load(f, encoding='bytes')
        X = datadict[b'data']
        Y = datadict[b'labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0, 2, 3, 1).astype("float")
        Y = np.array(Y)
        return X, Y

def load_CIFAR10(dirpath="/data/project/FoolingDetection/cifar-10-batches-py"):
    """Load all batches of the CIFAR-10 dataset from a directory"""
    xs = []
    ys = []
    for b in range(1, 5):
        f = os.path.join(dirpath, 'data_batch_%d' % (b,))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    Xte, Yte = load_CIFAR_batch(os.path.join(dirpath, 'test_batch'))
    return Xtr, Ytr, Xte, Yte

In [8]:
x_train, y_train, x_test, y_test = load_MNIST()
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

## Stop early!

In [9]:
early_stopping = tf.keras.callbacks.EarlyStopping(
  monitor="val_loss",
  patience=3,  
  restore_best_weights=True  
)

## Some Hyperparameters

In [10]:
### Hyperparameters
# Instantiate an optimizer, initializer, regularizer, loss_fn
optimizer = tf.keras.optimizers.Adam()
initializer = tf.keras.initializers.HeNormal()
regularizer = tf.keras.regularizers.L2(1e-4)
loss_fn = tf.losses.SparseCategoricalCrossentropy()

## Model

In [11]:
# Define a distribution strategy that uses only GPUs 0 and 1
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"])

def get_model_raw():
    with strategy.scope():
        ### Hyperparameters
        # Instantiate an optimizer, initializer, regularizer, loss_fn
        optimizer = tf.keras.optimizers.Adam()
        initializer = tf.keras.initializers.HeNormal()
        regularizer = tf.keras.regularizers.L2(1e-4)
        loss_fn = tf.losses.SparseCategoricalCrossentropy()
        metrics=tf.keras.metrics.SparseCategoricalAccuracy()

        # Alternative model from kaggle:
        model = keras.Sequential()

        model.add(layers.Conv2D(img_dim_x, (3,3), padding='same', activation='relu', input_shape=(img_dim_x,img_dim_y,img_dim_z)))
        model.add(layers.BatchNormalization())
        model.add(layers.Conv2D(img_dim_x, (3,3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2,2)))
        model.add(layers.Dropout(0.3))

        model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2,2)))
        model.add(layers.Dropout(0.5))

        model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2,2)))
        model.add(layers.Dropout(0.5))

        model.add(layers.Flatten())
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(10))
        model.add(layers.Activation('softmax'))
        
        model.compile(loss=loss_fn, optimizer='adam', metrics=[metrics])
        return model
model = get_model_raw()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/

In [12]:
# model.fit(x_train, y_train, epochs=2, batch_size=128, validation_data=(x_test,y_test), callbacks=callbacks)
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test,y_test), callbacks=[early_stopping])

Epoch 1/10


2023-05-25 15:52:38.801940: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


469/469 [==============================] - 24s 27ms/step - loss: 0.3967 - sparse_categorical_accuracy: 0.8799 - val_loss: 1.5189 - val_sparse_categorical_accuracy: 0.5548
Epoch 2/10
469/469 [==============================] - 11s 23ms/step - loss: 0.0921 - sparse_categorical_accuracy: 0.9721 - val_loss: 0.0262 - val_sparse_categorical_accuracy: 0.9911
Epoch 3/10
469/469 [==============================] - 11s 23ms/step - loss: 0.0663 - sparse_categorical_accuracy: 0.9798 - val_loss: 0.0215 - val_sparse_categorical_accuracy: 0.9927
Epoch 4/10
469/469 [==============================] - 11s 23ms/step - loss: 0.0531 - sparse_categorical_accuracy: 0.9837 - val_loss: 0.0238 - val_sparse_categorical_accuracy: 0.9928
Epoch 5/10
469/469 [==============================] - 11s 23ms/step - loss: 0.0467 - sparse_categorical_accuracy: 0.9857 - val_loss: 0.0254 - val_sparse_categorical_accuracy: 0.9923
Epoch 6/10
469/469 [==============================] - 11s 24ms/step - loss: 0.0436 - sparse_categoric

# Comparison 

## Naive approach: Take two images (or heatmap) (with same dimensions) and calculate different similarity metrics for them

In [1]:
from scipy.spatial import distance
from skimage.metrics import structural_similarity as ssim
import matplotlib.image as mpimg
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def euclidean_distance(im1, im2):
    im1 = im1.flatten()
    im2 = im2.flatten()
    return np.linalg.norm(im1 - im2)

def manhattan_distance(im1, im2):
    im1 = im1.flatten()
    im2 = im2.flatten()
    return distance.cityblock(im1, im2)
  
def minkowski_distance(im1, im2, p): # p=1 <-> Manhatten, p=2 Euclidian
    im1 = im1.flatten()
    im2 = im2.flatten()
    return distance.minkowski(im1, im2, p)

def chebyshev_distance(im1, im2):
    im1 = im1.flatten()
    im2 = im2.flatten()
    return distance.chebyshev(im1, im2)

def cosine_sim(im1, im2):
    im1 = im1.flatten()
    im2 = im2.flatten()
    return cosine_similarity(im1.reshape(1, -1), im2.reshape(1, -1))

def compute_ssim(im1, im2, multichannel=True):
    return ssim(im1, im2, multichannel=multichannel)

def run_all_distance_metrics(p1, p2):
  minkowski_p = 3
  return {"euclidean:": euclidean_distance(p1,p2),
          "manhatten:": manhattan_distance(p1,p2),
          "minkowski-{}".format(minkowski_p): minkowski_distance(p1, p2, minkowski_p),
          "chebyshev": chebyshev_distance(p1,p2),
          "cosine sim" : cosine_sim(p1,p2)
          }

# Running various techniques on more data

## Run the different explainers

In [5]:
# Function to create a dict with key=class values=data
def create_dict_classes_datapoints(x_test, y_test):
    class_dict = {}
    for i in range(len(x_test)):
        key = y_test[i]
        if key in class_dict:
            class_dict[key].append(x_test[i])
        else:
            class_dict[key] = [x_test[i]]
    return class_dict

# # Define sorted labels 
# unique_sorted_labels = sorted(set(map(str, y_test)), key=lambda x: (not x.isdigit(), int(x) if x.isdigit() else x))
# images_by_class = create_dict_classes_datapoints(x_test, y_test)

In [4]:
def run_explainers(x_in, y_in, model, NUM_IMAGES=300):

    explainer_grad = GradCAM()
    grids_gradcam = []
    grids_gradcam_guided = []
    explainer_ig = IntegratedGradients()
    grids_ig = []
    explainer_g_t_i = GradientsInputs()
    grids_g_t_i = []
    explainer_vgrads = VanillaGradients()
    grids_vgrads = []
    explainer_occlusion = OcclusionSensitivity()
    grids_occlusion = []

    # Iterate over the test_dataset for NUM_IMAGES. Run all the explanations using tf-explain and store the results in lists to be accessed later
    for index, (x_example, y_example) in enumerate(zip(x_in, y_in)):
      if index == NUM_IMAGES:
        break
      print("image: {}".format(index))
      x_example = ([x_example], None)
      y_example_index = int(y_example)

      grids_gradcam.append(explainer_grad.explain(x_example, model, class_index=y_example_index, use_guided_grads=False))
      grids_gradcam_guided.append(explainer_grad.explain(x_example, model, class_index=y_example_index, use_guided_grads=True))
      grids_ig.append(explainer_ig.explain(x_example, model, class_index=y_example_index))
      grids_g_t_i.append(explainer_g_t_i.explain(x_example, model, class_index=y_example_index))
      grids_vgrads.append(explainer_vgrads.explain(x_example, model, class_index=y_example_index))
      grids_occlusion.append(explainer_occlusion.explain(x_example, model, class_index=y_example_index, patch_size=7))
    
    return [(grids_gradcam, "gradcam"), 
              (grids_gradcam_guided, "gradcam_guided"), 
              (grids_ig, "ig"), 
              (grids_g_t_i, "g_t_i"), 
              (grids_vgrads, "vgrads"), 
              (grids_occlusion, "occlusion")]

In [6]:
### Now store the grids on the local disk so we do not have to re-run this everytime


# grid_list needs to match definiton in run_explainations()
def save_grids_to_disk(grids_list, save_dir, dataset, model, attack="correct", override=False):

    # Define save directory
    save_dir = os.path.join(save_dir, dataset, model, attack)
    
    if os.path.exists(save_dir) and not override:
        raise Exception("The directory already exists and `override=False` was passed. Please make sure that you want to do this, this will delete any previous results.")
    
    # Iterate over all grids and save them to corresponding subfolder
    for grids, subfolder in grids_list:
        subfolder_path = os.path.join(save_dir, subfolder)

        # Create subdirectory if it does not exist
        if not os.path.exists(subfolder_path):
            os.makedirs(subfolder_path)

        # Save each grid as an image
        for i, grid in enumerate(grids):
            plt.imshow(grid, cmap='hot', interpolation='nearest')
            plt.savefig(os.path.join(subfolder_path, f"{i}.png"))
            plt.close()

In [7]:
# New function to run the metrics which supports batches of images
def run_all_distance_metrics_batch(P1, P2):
    minkowski_p = 3
    n = len(P1)
    result = {"euclidean": 0,
              "manhattan": 0,
              "minkowski-{}".format(minkowski_p): 0,
              "chebyshev": 0,
              "cosine sim": 0}
    for p1, p2 in zip(P1, P2):
        result["euclidean"] += euclidean_distance(p1,p2)
        result["manhattan"] += manhattan_distance(p1,p2)
        result["minkowski-{}".format(minkowski_p)] += minkowski_distance(p1, p2, minkowski_p)
        result["chebyshev"] += chebyshev_distance(p1,p2)
        result["cosine sim"] += cosine_sim(p1,p2)
    for key in result:
        result[key] /= n
    return result